In [15]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from local_settings import postgresql as settings

In [16]:
import os
print(os.getcwd())

d:\Daniel\data_science\portfolio_projects\food_carbon_fp


In [17]:
def get_engine(user, passwd, host, port, db):
    url = f"mssql+pyodbc://{user}:{passwd}@{host},{port}/{db}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(url, pool_size= 50, echo= False)

    return engine

In [18]:
engine = get_engine(
    user= settings['pguser'],
    passwd=settings['pgpasswd'],
    host=settings['pghost'],
    port=settings['pgport'],
    db=settings['pgdb']
)

connection = engine.connect()

In [19]:
from sqlalchemy import inspect, MetaData, Table


inspector = inspect(engine)
print(inspector.get_table_names())

['food_cfp', 'food_cfp_2018']


In [59]:
metadata = MetaData()

food_cfp_2018 = Table('food_cfp_2018', metadata, autoload= True, autoload_with= engine)

print(food_cfp_2018.columns.keys())

['Country', 'Product', 'Metric', 'Value']


In [67]:
from sqlalchemy import select
import pandas as pd

stmt = select([food_cfp_2018])

query = "SELECT * FROM food_cfp_2018"

pd.read_sql_query(query, connection)

,Country,Product,Metric,Value
0,Albania,Milk - inc. Cheese,Kg CO2/person/year,432.619995
1,Albania,Pork,Kg CO2/person/year,38.509998
2,Albania,Fish,Supplied for Consumption (kg/person/year),12.450000
3,Albania,Soybeans,Kg CO2/person/year,0.000000
4,Albania,Fish,Kg CO2/person/year,6.150000
...,...,...,...,...
3245,Zimbabwe,Rice,Supplied for Consumption (kg/person/year),10.090000
3246,Zimbabwe,Wheat & Wheat Products,Kg CO2/person/year,5.970000
3247,Zimbabwe,Poultry,Kg CO2/person/year,5.340000
3248,Zimbabwe,Poultry,Supplied for Consumption (kg/person/year),4.970000


##### Exploratory Data Analysis

In [68]:
# Evaluated products and metrics

stmt = select(food_cfp_2018.columns.Product.distinct(), food_cfp_2018.columns.Metric)
stmt = stmt.order_by(food_cfp_2018.columns.Product)

pd.DataFrame(connection.execute(stmt), 
             columns= [food_cfp_2018.columns.keys()[1], food_cfp_2018.columns.keys()[2]])

,Product,Metric
0,Beef,Supplied for Consumption (kg/person/year)
1,Beef,Kg CO2/person/year
2,Difference between Animal Products/person/year...,Difference between Animal Products/person/year...
3,Eggs,Kg CO2/person/year
4,Fish,Kg CO2/person/year
5,Fish,Supplied for Consumption (kg/person/year)
6,Lamb & Goat,Supplied for Consumption (kg/person/year)
7,Lamb & Goat,Kg CO2/person/year
8,Milk - inc. Cheese,Kg CO2/person/year
9,Milk - inc. Cheese,Supplied for Consumption (kg/person/year)


In [69]:
# Filtering information for the first country

query = """
    SELECT * 
        FROM food_cfp_2018
        WHERE Country LIKE 'Albania'
"""

pd.read_sql_query(query, connection)

,Country,Product,Metric,Value
0,Albania,Milk - inc. Cheese,Kg CO2/person/year,432.619995
1,Albania,Pork,Kg CO2/person/year,38.509998
2,Albania,Fish,Supplied for Consumption (kg/person/year),12.450000
3,Albania,Soybeans,Kg CO2/person/year,0.000000
4,Albania,Fish,Kg CO2/person/year,6.150000
5,Albania,Rice,Kg CO2/person/year,9.960000
6,Albania,Fish,Supplied for Consumption (kg/person/year),3.850000
7,Albania,Milk - inc. Cheese,Supplied for Consumption (kg/person/year),303.720001
8,Albania,Nuts inc. Peanut Butter,Supplied for Consumption (kg/person/year),4.360000
9,Albania,Lamb & Goat,Kg CO2/person/year,536.500000
